In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [3]:
living_pop = pd.read_csv('living_population/living_pop_neighborhood.csv')
living_pop

,adm_cd2,total_Apr_0,total_Apr_1,total_Apr_2,total_Apr_3,total_Apr_4,total_Apr_5,total_Apr_6,total_Apr_7,total_Apr_8,...,male_avg_23,female_avg_23,total_avg_avg,male_avg_avg,female_avg_avg,gu,max,min,lat,lng
0,11110515,15268,15310,15303,15300,15474,15579,16285,18373,20585,...,7390.208333,7946.166667,17081.635417,8515.444444,8870.090278,11110,24665.0,13533.0,37.584768,126.969029
1,11110530,21557,21028,20605,21748,22767,22054,24529,31030,42664,...,9692.625000,9820.583333,34088.340278,17023.319444,17183.281250,11110,57622.0,14441.0,37.573898,126.969922
2,11110540,4643,4582,4628,4632,4658,4773,5161,6451,7760,...,2404.750000,2294.750000,7252.423611,3622.053819,3501.878472,11110,13209.0,4392.0,37.586045,126.981532
3,11110550,14292,13985,13915,13874,13242,13302,13524,13965,14343,...,6756.500000,7489.500000,13851.381944,6913.385417,7677.402778,11110,16767.0,11187.0,37.595435,126.964300
4,11110560,21776,22122,22421,22484,22887,23129,23092,22077,20993,...,10828.208333,11966.166667,20080.697917,10018.236111,11111.357639,11110,23792.0,16358.0,37.613675,126.968862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,11740650,32598,32270,32175,32062,31481,31264,30783,29990,27737,...,15975.625000,16812.000000,29058.875000,14487.998264,15590.548611,11740,35893.0,23545.0,37.534335,127.127824
414,11740660,29389,30315,30466,30474,31297,31076,30539,29609,28026,...,14139.208333,14645.333333,28415.260417,14164.939236,14766.458333,11740,31525.0,25451.0,37.528406,127.133657
415,11740685,58218,57970,57960,57868,57554,56646,55028,52339,49238,...,29577.291667,31250.250000,54163.024306,27003.819444,29005.982639,11740,61714.0,44958.0,37.539865,127.147347
416,11740690,6211,6121,6100,6123,6025,6153,6176,6652,6776,...,3541.458333,3546.166667,7149.250000,3561.274306,3549.218750,11740,9523.0,4333.0,37.522880,127.140035


In [4]:
neighborhood_code = pd.DataFrame(gpd.read_file('census/HangJeongDong_ver20200101.geojson'))[['adm_nm','adm_cd','adm_cd2']]
neighborhood_code = neighborhood_code.loc[neighborhood_code['adm_nm'].str.find('서울특별시') == 0]
neighborhood_code['gu'] = neighborhood_code['adm_nm'].apply(lambda x:x.split()[-2])
neighborhood_code['dong'] = neighborhood_code['adm_nm'].apply(lambda x:x.split()[-1])
neighborhood_code = neighborhood_code.drop('adm_nm',axis=1)
neighborhood_code['adm_cd2'] = neighborhood_code['adm_cd2'].astype(int)//100
neighborhood_code.head()

,adm_cd,adm_cd2,gu,dong
0,1101053,11110530,종로구,사직동
1,1101054,11110540,종로구,삼청동
2,1101055,11110550,종로구,부암동
3,1101056,11110560,종로구,평창동
4,1101057,11110570,종로구,무악동


In [5]:
census = pd.read_csv('census/census(2019).csv')
census = census.iloc[2:,:]
census = census.rename(columns={'기간':'time','자치구':'gu','행정동':'dong','인구':'census_pop'})
census = census.loc[census['dong']!='소계']
census['census_pop'] = census['census_pop'].str.replace(',','').astype(int)

In [6]:
census = census.merge(right= neighborhood_code[['gu','dong','adm_cd2']],left_on = ['gu','dong'], right_on = ['gu','dong'], how='inner')
census_pivot = census.pivot(index='adm_cd2', columns='time', values='census_pop')

In [7]:
census_pivot.to_csv('census_pop.csv')

In [7]:
living_pop  = living_pop.merge(right=census_pivot, left_on='adm_cd2', right_index=True, how='left')

In [8]:
living_pop.to_csv('pop.csv',index=False)